# env setting

In [68]:
import sys

from numpy.core.fromnumeric import mean
sys.path.append("/Users/huangguanlun/pyProjects/quotoFuture/")
sys.path.append("/Users/huangguanlun/pyProjects/quotoFuture/qfVenv/lib/python3.8/site-packages/")

In [69]:
import simContainer
import numpy as np
from scipy.linalg import cholesky, inv

In [70]:
FLOAT_FORMATTER="{:.5f}".format
np.set_printoptions(formatter={'float_kind':FLOAT_FORMATTER})

# parameter and container declaration

In [71]:
S0Arr = [100, 0.5]
muArr = [0.5, 0.5]
riskFreeRate = 0.03
timePeriod = 0.5
timePartitionCnt = 200
qArr = [0, 0]
sigmaArr = [0.03, 0.05]
corrMatrx = [[1, 0.4], [0.4, 1]]
simCnt = 5000
repeatCnt = 10

In [72]:
test = simContainer.pathSim(
    S0Arr= S0Arr,
    muArr = muArr,
    riskFreeRate= riskFreeRate,
    timePartitionCnt= timePartitionCnt,
    timePeriod= timePeriod,
    qArr= qArr,
    sigmaArr= sigmaArr,
    corrMatrx= corrMatrx,
    simCnt= simCnt,
    repeatCnt= repeatCnt
)

In [73]:
riskFreeRate = test.riskFreeRate

In [74]:
test.Z0Container.container.shape

(10, 5000, 200, 2)

# path simulation

In [75]:
test.var_cov()
test.muShift()
test.lnS0Append()
test.cumsum()

# carry cost computation

In [76]:
posValSeries = test.stSeries[..., 0] / test.stSeries[..., 1]
plSeries = np.diff(posValSeries, axis = 2)
deltaTCnt = np.ones_like(plSeries) * np.arange(plSeries.shape[2])[::-1]
extraCarryCostSeries= plSeries * np.exp(riskFreeRate * deltaTCnt * (timePeriod / timePartitionCnt))
extraCarryCost = extraCarryCostSeries.sum(axis = 2)

In [77]:
meanExtraCarryCost = extraCarryCost.mean(axis = 1).mean(axis = 0)

In [78]:
meanExtraCarryCost

0.08900665149449749

In [79]:
initCost = (S0Arr[0] / S0Arr[1]) * np.exp(riskFreeRate * timePeriod)

In [80]:
initCost

241.24604988419614

In [81]:
simAns = meanExtraCarryCost + initCost

In [82]:
var_T = test.sigmaArr * np.power(test.timePeriod, 0.5).reshape(1, -1)
var_T

array([[0.02121, 0.03536]])

In [83]:
test.corrMtrx

array([[1.00000, 0.40000],
       [0.40000, 1.00000]])

In [84]:
covMtrx = test.corrMtrx * np.tensordot(var_T, var_T, axes= (0, 0))
covMtrx

array([[0.00045, 0.00030],
       [0.00030, 0.00125]])

In [85]:
formulaSol = S0Arr[0] * np.exp(riskFreeRate * timePeriod) * np.exp(covMtrx[0][1])

In [86]:
S0Arr[0] * np.exp(riskFreeRate * timePeriod)

120.62302494209807

In [87]:
formulaSol

120.65921727815966

In [88]:
stdErr = extraCarryCost.mean(axis = 1).std()

In [89]:
stdErr

0.005054530757104987

In [90]:
formulaSol / S0Arr[1], simAns

(241.31843455631932, 241.33505653569063)

In [91]:
simAns - 2 * stdErr, formulaSol / S0Arr[1], simAns + 2 * stdErr

(241.3249474741764, 241.31843455631932, 241.34516559720484)

In [92]:
formulaSol, simAns * S0Arr[1]

(120.65921727815966, 120.66752826784531)